In [1]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
import time
import json
from selenium.webdriver.chrome.service import Service
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

### GET informations about each 35 destinations

In [2]:
list_35_destinations = [
#"Mont Saint Michel",
#"St Malo",
#"Bayeux",
#"Le Havre",
#"Rouen",
#"Paris",
#"Amiens",
#"Lille",
#"Strasbourg",
#"Chateau du Haut Koenigsbourg",
#"Colmar",
#"Eguisheim",
#"Besancon",
#"Dijon",
# "Annecy",
# "Grenoble",
# "Lyon",
# "Gorges du Verdon",
# "Bormes les Mimosas",
# "Cassis",
# "Marseille",
# "Aix en Provence",
# "Avignon",
# "Uzes",
# "Nimes",
# "Aigues Mortes",
# "Saintes Maries de la mer",
# "Collioure",
# "Carcassonne",
# "Ariege",
# "Toulouse",
# "Montauban",
# "Biarritz",
# "Bayonne",
# "La Rochelle"
]


In [ ]:

for city in list_35_destinations:

    time.sleep(30)
    path=Service("C:\\Program Files (x86)\\Google\\Chrome\\Application\\chromedriver\\chromedriver.exe")
    browser = webdriver.Chrome(chrome_options=options, service=path)
    browser.get("https://www.booking.com/")
    browser.maximize_window()
    time.sleep(5)

    try:
        rgpd_button = browser.find_element(By.XPATH, '//*[@id="onetrust-reject-all-handler"]')
        rgpd_button.click()
    except:
        rgpd_button = browser.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
        rgpd_button.click()
    time.sleep(3)

    text_box = browser.find_element(By.NAME,'ss')
    submit_button = browser.find_element(By.XPATH,'//*[@id="frm"]/div[1]/div[4]/div[2]/button')

    text_box.send_keys(city)
    submit_button.click()
    time.sleep(8)

    ##### SORT RESULT WITH STARS DESCENDING ORDER ####

    # Be Carefull with this piece of code, it works but XPATH of button change regularly
    # and needs to be updated manually

    # sort_result_button = WebDriverWait(browser, timeout=20).until(lambda d: d.find_element(By.XPATH, '//*[@id="search_results_table"]/div[2]/div/div/div/div[2]/span/button'))
    # #sort_result_button = browser.find_element(By.XPATH, '//*[@id="search_results_table"]/div[2]/div/div/div/div[2]/span/button')
    # sort_result_button.click()                          
    # time.sleep(3)

    # try:
    #     button_step_1 = browser.find_element(By.XPATH, '/html/body/div[18]/div/div/div/div/ul/li[3]')
    #     button_step_2 = button_step_1.find_element(By.TAG_NAME, 'button').click()                      

    # except NoSuchElementException:
    #     etoiles_ordre_decroissant_button = browser.find_element(By.XPATH, '//*[@id="__bui-c3-1"]/div/div/div/div/ul/li[3]/button') 
    #     etoiles_ordre_decroissant_button.click()
          
    # time.sleep(5)
    #################################################


    #### INIT LOOP TO SCRAP 25 RESULTS FOR THIS CITY NAME ####
    list_results = []
    count=1


    property_cards = browser.find_elements(By.CSS_SELECTOR, 'div[data-testid="property-card"]')

    for property in property_cards:
        new_result = {}

        print(count)

        try:
            name = property.find_element(By.CSS_SELECTOR,'div[data-testid="title"]').text
        except:
            name = 'N/A'
        print(name)

        try:
            link = property.find_element(By.CSS_SELECTOR,'a[data-testid="title-link"]').get_attribute('href')
        except:
            link = 'N/A'
        print(link)

        try:
            review_score, _, review_count = property.find_element(By.CSS_SELECTOR, '[data-testid="review-score"]').text.split('\n')
        except:
            review_score = 'N/A' 
            review_count = 'N/A'
        print(review_score, review_count)

        try:
            area = property.find_element(By.CSS_SELECTOR, '[data-testid="address"]').text
        except:
            area = 'N/A'
        print(area)
        

        ##### RECORDING RESULTS AND OPEN A NEW WINDOW TO SCRAP MORE INFORMATIONS ########    

        new_result['name'] = name
        new_result['link'] = link
        new_result['review_score'] = review_score
        new_result['review_count'] = review_count
        #new_result['price'] = price
        new_result['area'] = area


        ##################

        # init browser2
        time.sleep(2)
        browser2 = webdriver.Chrome(chrome_options=options, service=path)
        browser2.get(link)
        browser2.maximize_window()
        time.sleep(10)

        #RGPD Button
        try:
            time.sleep(3)
            rgpd_button = browser2.find_element(By.XPATH, '//*[@id="onetrust-reject-all-handler"]')
            rgpd_button.click()
        except:
            time.sleep(3)
            rgpd_button = browser2.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
            rgpd_button.click()
        time.sleep(3)

        try:
            address = browser2.find_element(By.XPATH, '//*[@id="showMap2"]/span[1]').text
        except:
            address = 'N/A'

        new_result['address'] = address
        print(address)
        time.sleep(1)

        try:
            coordinates_gps = browser2.find_element(By.XPATH, '//*[@id="showMap2"]/span[1]').get_attribute('data-bbox')
        except:
            coordinates_gps = 'N/A'
            
        new_result['coordinates_gps'] = coordinates_gps 
        print(coordinates_gps)
        print()
        time.sleep(1)

        try:
            button_see_more = browser2.find_element(By.XPATH, '//*[@id="basiclayout"]/div[1]/div[2]/div/div/div[1]/div[1]/div[1]/div[2]/a').click()
            time.sleep(2)
        except:
            button_see_more = 'N/A'
        
        try:
            description = browser2.find_element(By.XPATH, '//*[@id="basiclayout"]/div[1]/div[2]/div/div/div[1]/div[1]/div[1]').text
        except:
            description = 'N/A'

        #### RECORDING ADDITIONNAL RESULTS AND CLOSING BROWSER 2 ####

        new_result['description'] = description
        time.sleep(1)

        # close browser 2
        browser2.close()

        ##################


        # add all informations about 25 hotels in a list
        list_results.append(new_result)

        count +=1
        time.sleep(3)
        ### END OF RESULTS PER CITY LOOP ###




    # create a Dataframe for this CITY and export as a csv file
    df = pd.DataFrame(list_results)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    df.to_csv(f'save_output/{timestr}-{city}.csv', index=False)

    # create  and record a json file containing this list of dict
    # Serializing json
    json_object = json.dumps(list_results)
    # Writing to sample.json
    with open(f"save_output/{timestr}-{city}.json", "w") as outfile:
        outfile.write(json_object)
    
    time.sleep(15)
    browser.close()
    # END OF CITY LOOP

